In [2]:
import pandas as pd
import numpy as np
from pydantic import BaseModel
from openai import OpenAI
import os
import json

In [3]:
# load data
df_raw = pd.read_excel("data/raw/atmospere_data.xlsx")

In [4]:
# Creating a list of tuples (index, original row number, column name, value)
records = []
counter = 1
for index, row in df_raw.iterrows():
    for col in df_raw.columns:
        records.append((counter, index + 1, col, row[col]))  # index + 1 to make it 1-based
        counter += 1

# Convert the list of tuples to a DataFrame
df = pd.DataFrame(records, columns=['Index', 'Document', 'ColumnName', 'Value'])
df = df.drop(columns=['Index'])

In [5]:
df

,Document,ColumnName,Value
0,1,working_atmosphere_comment,"Herzlich, freundlich, hilfsbereit und voralle..."
1,1,leadership_behavior_comment,"Ich habe einer super Mentorin , die mir die Mö..."
2,1,team_spirit_comment,Sehr hilfsbereit und auch neben dem Job trinkt...
3,1,interesting_tasks_comment,Bei der Beratung steht der Mensch im Mittelpun...
4,1,communication_comment,"Freundlich, konstruktiv und höflich."
...,...,...,...
1395,200,team_spirit_comment,Die Kollegen und der Zusammenhalt ist klasse!
1396,200,interesting_tasks_comment,Die Arbeit macht Spaß und ist vielseitig...es ...
1397,200,communication_comment,"Allgemein gut, nur manchmal fehlt die Transpar..."
1398,200,working_conditions_comment,"Systeme hängen ab und an, sonst alles top! Küc..."


In [6]:

# set function output format
class StepByStepAIResponse(BaseModel):
    topic: str
    short_sumary: str
    key_aspect: str
    key_word: str
    positve: bool
    rating: int
    assagekraft: int
    sarcasm_rating: bool
    
# set openai api key
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)


In [7]:

def call_open_ai(client: OpenAI, statement_df: pd.Series) -> dict:
    
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": statement_df['Value'],
            }
        ],
        functions=[
            {
            "name": "beschreibe_jobportal_bewertung",
            "description": """Beschreibe eine Jobportal-Bewertung mit einer übergeordneten Topic, 
            einer extrem kurzen Zusammenfassen, dem key-aspect, dem key-word, 
            einem positivem or negativem Sentiment, einer geschätzen Abeitgeber-Attraktivitäts Bewertung 0-10, 
            einer geschätzen Aussagekraft der Bewertung 0-10 (sei kritisch!), einer Sarkasmus Einschätzung 0-10""",
            "parameters": StepByStepAIResponse.model_json_schema()
            }
        ],
        function_call={"name": "beschreibe_jobportal_bewertung"}
    )

    completion_value = completion.choices[0].message.function_call.arguments
    completion_dict = json.loads(completion_value)
    completion_dict['value'] = statement_df['Value']
    completion_dict['document'] = statement_df['Document']
    completion_dict['column_name'] = statement_df['ColumnName']
    
    return completion_dict
    


In [8]:
results = []
for index, row in df.iterrows():
    
    # ignore missing values
    if row['Value'] is np.nan:
        continue
    
    # call open ai
    result = call_open_ai(client, row)
    results.append(result)
    
    print(f"{index}: {result['key_aspect']}")
    
    if index % 20 == 0:
        result_df = pd.DataFrame(results)
        result_df.to_excel("data/openai_results.xlsx")

0: Mitarbeiterinteraktion
1: Mentoring
2: Kollegiales Miteinander
3: Vielseitigkeit
4: Umgangston


In [ ]:
result_df = pd.DataFrame(results)
result_df.to_excel("data/openai_results.xlsx")